In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from xwp_rotation.rotate import pyvips_rotate, rot_matrix
from xwp_rotation.utils import get_valid_locs

import petsc4py
petsc4py.init()
from petsc4py import PETSc

np.set_printoptions(precision=2)

In [2]:
size = 64
angle = 30
rot_mat = rot_matrix(size,angle)
idx,idy = np.where(rot_mat!=0)
print(np.count_nonzero(rot_mat)/(size**4))

0.002442002296447754


In [3]:
A = PETSc.Mat().create(PETSc.COMM_WORLD)
A.setSizes([size**2,size**2])
A.setType('seqaij')
A.setFromOptions()
A.setUp()

In [4]:
Istart, Iend = A.getOwnershipRange()
for I in range(len(idx)):
    A[idx[I],idy[I]] = rot_mat[idx[I],idy[I]]
A.assemblyBegin()
A.assemblyEnd()

In [5]:
viewer = PETSc.Viewer().createBinary('rot.dat', 'w')
viewer(A)